# Real-time ticker for SBUX prices

*******************************************************************************
    #Created at: 26/07/2017
    #Author: Michelle Malone
    #Purpose: Using a Url to generate a live ticker stream of SBUX share prices. 
********************************************************************************

In [ ]:
# importing libraries to be used

import bs4 as bs 
from urllib.request import Request, urlopen
import pyodbc
from datetime import datetime
import time

In [ ]:
#Connect to the SQL server
conn = pyodbc.connect(r'DSN=ticker')
curr= conn.cursor()

#Define a function
def priceinformation():   
    req=Request('https://uk.investing.com/equities/starbucks-corp', headers={'User-Agent': 'Mozilla/5.0'})
    
#Read through the URL which contains SBUX ticker    
    webpage = urlopen(req).read()  
    s = bs.BeautifulSoup(webpage,'html.parser')

#Pass through the html and search for 'class' and the string of words      
    Priceline = s.find_all('span', {'class':'arial_26 inlineblock pid-6500-last'})
    Dateline = s.find_all('span', {'class': 'bold pid-6500-time'})

#for loop: iterate through the Dateline, if ':' doesn't exist a -1 will be produced.
#Therefore only continue the loop if this value is greater than zero
    for e in Dateline:
        codeline = str(e)
        if codeline.find(':')>0:
            position= codeline.find(':')
            Time =codeline[position -2: position +6]
#transform the string into a datetime format
            FinalTime =datetime.strptime(str(datetime.now()),'%Y-%m-%d %H:%M:%S.%f') 
            daytime = 1   #daytime=1: daytime 
            
            
        else: 
            daytime=0    #daytime=0: nighttime 
       
         
    for e in Priceline:   #for loop: iterate through the Priceline 
        if daytime == 1:
            codeline =str(e)
            position = codeline.find('.')   #look for '.', is the dp in the price e.g 58.64 
            Price = codeline[position -2:position +3] 
            
    
    if daytime == 1:     #Will only insert into sql table during daytime hours and when the market is open   
        curr.execute('insert into ticker (Uk_time, [Price]) values(?,?)', FinalTime, Price)
        conn.commit()

In [88]:
#while loop: loop through the function priceinformation, every 60 seconds
while True:
    priceinformation()
    time.sleep(60)

KeyboardInterrupt: 